In [72]:
from src import setup
# This prevents to restart the kernel whenever an external file is edited
%load_ext autoreload
%autoreload 2

# Scripts: see below (computation management)
RUN_SCRIPTS = True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Introduction

## Computation management
We run all the bash scripts with the function below. Idea: two separate procedures, depending on wether the computations have already been done locally. Principle:

1. Call of the custom function `run(path)` and hide output by default.
    * If `RUN_SCRIPTS` == False, simply do nothing
    * If `RUN_SCRIPTS` == True, run the script
1. `cat` the log file.

Example: while coding or after importing source code, put `RUN_SCRIPTS = True`. While re-running a notebook (ex: check that the code runs), put `RUN_SCRIPTS = False`.

In [73]:
def run(path=None, func=None):
    if RUN_SCRIPTS == False: return
    if path is not None:
        !./pwd
    if func is not None:
        func()

## Overview on raw data

### Files

In [12]:
path = setup.PATH_HOST_RAW_DATA
!ls -lh $path

total 2.6G
-rwxr-xr-x 1 mazeller SV-BA4-unit  245 Aug  9 16:11 dictionary_wes_vcf.txt
-rwxr-xr-x 1 mazeller SV-BA4-unit 2.2G Aug  9 16:11 hbv_gilead.bed
-rwxr-xr-x 1 mazeller SV-BA4-unit 431M Aug  9 16:11 hbv_gilead.bim
-rwxr-xr-x 1 mazeller SV-BA4-unit  18K Aug  9 16:11 hbv_gilead.fam


Those are **plink binary files**. 

### Dictionnary of the data

In [13]:
!cat $path/dictionary_wes_vcf.txt

Info about WES_VCF by Ondrej.Podlaha@gilead.com on Dec 12 2018.

1. Human reference genome: GRCh37.87
2. Variant calling was performed using GATK (version nightly-2016-08-27-g667f78b; HaplotypeCaller)
3. dbSNP v138 was used by HaplotypeCaller
 


### Summary of data - plink2

We simply run plink with the `--bfile` option to get an overview on the data

In [15]:
def plink_summary():
    !plink2 

/usr/bin/sh: /home/mazeller/opt/x86_E5v2_IntelIB/venv-gcc/bin:/ssoft/spack/paien/v2/opt/spack/linux-rhel7-x86_E5v2_IntelIB/gcc-6.4.0/python-3.6.5-wq64mct6tytbe3dp5iv65jlwd7rn7wvd/bin:/ssoft/spack/paien/v2/opt/spack/linux-rhel7-x86_E5v2_IntelIB/gcc-4.8.5/gcc-6.4.0-5lz2sine3ujgoc7aswaexuinafuepktx/bin:/usr/lib64/qt-3.3/bin:/usr/lib64/ccache:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/usr/lpp/mmfs/bin:/home/mazeller/.local/bin:/home/mazeller/bin: No such file or directory


<br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br />